## Building question-and-answer pipelines for complex knowledge bases

#### Build an indexing pipeline from Haystack's documentation

In [ ]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy



document_store = InMemoryDocumentStore()

link_fetcher = LinkContentFetcher()
converter = HTMLToDocument()
splitter = DocumentSplitter(split_length=150, split_overlap=5, split_by="sentence")
embedder = SentenceTransformersDocumentEmbedder()
writer = DocumentWriter(document_store=document_store, 
                        policy=DuplicatePolicy.OVERWRITE)

indexing_pipeline = Pipeline()
indexing_pipeline.add_component("link_fetcher", link_fetcher)
indexing_pipeline.add_component("converter", converter)
indexing_pipeline.add_component("splitter", splitter)
indexing_pipeline.add_component("embedder", embedder)
indexing_pipeline.add_component("writer", writer)

indexing_pipeline.connect("link_fetcher", "converter")
indexing_pipeline.connect("converter", "splitter")
indexing_pipeline.connect("splitter", "embedder")
indexing_pipeline.connect("embedder", "writer")

In [21]:
indexing_pipeline.draw("indexing_complex_knowledge.png")

![](./indexing_complex_knowledge.png)

##### Populate the document store with relevant links

In [ ]:
indexing_pipeline.run(data={"link_fetcher":{"urls": ["https://haystack.deepset.ai/integrations/elasticsearch-document-store",
                                                    "https://haystack.deepset.ai/tutorials/27_first_rag_pipeline/",
                                                     "https://haystack.deepset.ai/tutorials/28_structured_output_with_loop/",
                                                     "https://haystack.deepset.ai/tutorials/36_building_fallbacks_with_conditional_routing/",
                                                     "https://haystack.deepset.ai/tutorials/40_building_chat_application_with_function_calling/",
                                                     "https://haystack.deepset.ai/tutorials/42_sentence_window_retriever/",
                                                     "https://haystack.deepset.ai/tutorials/41_query_classification_with_transformerstextrouter_and_transformerszeroshottextrouter/",
                                                     "https://haystack.deepset.ai/tutorials/31_metadata_filtering/",
                                                     "https://haystack.deepset.ai/tutorials/30_file_type_preprocessing_index_pipeline/",
                                                    "https://haystack.deepset.ai/tutorials/39_embedding_metadata_for_improved_retrieval/",
                                                    "https://haystack.deepset.ai/tutorials/29_serializing_pipelines/",
                                                    "https://haystack.deepset.ai/tutorials/34_extractive_qa_pipeline/",
                                                    "https://haystack.deepset.ai/tutorials/33_hybrid_retrieval/",
                                                    "https://haystack.deepset.ai/tutorials/32_classifying_documents_and_queries_by_language/",
                                                    "https://haystack.deepset.ai/tutorials/35_evaluating_rag_pipelines/",
                                                    "https://haystack.deepset.ai/tutorials/37_simplifying_pipeline_inputs_with_multiplexer/"]}})

In [3]:
from dotenv import load_dotenv
import os

load_dotenv("./../.env")

open_ai_key = os.getenv("OPENAI_API_KEY")

### Answer questions using prompt templating

In [10]:
import torch

from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import OpenAIGenerator

######## Complete this section #############
prompt_template = """
You are an expert Python software engineer, you are asked to write code, 
explain code and you use the context provided to generate accurate and functional code along with clear explanations.
After you define a class, you also provide examples of using the class and its methods.
You must only use information from the given documents and cite the documents you used by mentioning their URL in the answer.
For example, begin your answer with ‘As stated in URL, ...’.
If the documents do not contain the answer to the question, say that ‘Answer is unknown.’
Context:
{% for doc in documents %}
    Document: {{ doc.content }} URL: {{ doc.meta['url'] }} \n
{% endfor %};
Question: {{query}}
\nAs stated in
"""
prompt_builder = PromptBuilder(prompt_template)
############################################
query_embedder = SentenceTransformersTextEmbedder()
retriever = InMemoryEmbeddingRetriever(document_store=document_store, top_k=2)
llm = OpenAIGenerator(model='gpt-4')

In [ ]:
pipeline = Pipeline()
pipeline.add_component(instance=query_embedder, name="query_embedder")
pipeline.add_component(instance=retriever, name="retriever")
pipeline.add_component(instance=prompt_builder, name="prompt_builder")
pipeline.add_component(instance=llm, name="llm")

pipeline.connect("query_embedder.embedding", "retriever.query_embedding")
pipeline.connect("retriever.documents", "prompt_builder.documents")
pipeline.connect("prompt_builder", "llm")

In [22]:
pipeline.draw("answer_generation_pipeline.png")

![](./answer_generation_pipeline.png)

In [ ]:
question = "Write a Haystack 2.0 pipeline that connects to Elastic Search and answers questions about its knowledge, the pipeline\
    should include a prompt template with instructions and the template should iterate over all documents in the \
        context, the pipeline should also include a feedback loop and guardrails to ensure that the answers are accurate."
result = pipeline.run(data={"query_embedder": {"text": question}, "prompt_builder": {"query": question}})
print(result['llm']['replies'][0])